## Exploration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import islice

import tkseem as tk
import gensim

/home/ellzo/.local/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
df_poems = pd.read_csv('all_poems.csv')
df_poems.head()

,poem_id,poem_link,poem_style,poem_text,poem_title,poet_cat,poet_id,poet_link,poet_name
0,21,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...,أنشودة المطر,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب
1,65546,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...,أقداح و أحلام,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب
2,65561,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة ...,هوى واحد !,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب
3,65554,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,اساطير من حشرجات الزمان نسيج اليد البالية رواه...,أساطير,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب
4,65550,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,والتف حولك ساعداي ومال جيدك في اشتهاء كالزهرة ...,اللقاء الأخير,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب


In [3]:
gensim.utils.simple_preprocess("إني امرؤ من خير عبس منصبا")

['إني', 'امرؤ', 'من', 'خير', 'عبس', 'منصبا']

In [4]:
indices = np.random.choice(df_poems.shape[0], size=1000, replace=False)

df_poems_sample = df_poems.loc[indices, ['poem_text']]
df_poems_sample.head()

,poem_text
19455,اما ابن يعقوب فاندي الوري واعلم القوم ولا امت...
10854,مالكثيب الحمي الي عقده مابال جرعاءه الي جرده ...
5770,اصاح اذا ما اتاك القضاء لم يقك الدرع والجوشن ...
31463,عبث النسيم بقده فتاودا وسري الحياء بخده فتورد...
51069,رمضان يا شهر الضياء الحر من اسر الظلام اطلق ب...


In [5]:
import csv

df_poems_sample.to_csv('data.txt', sep=' ', index=False, header=False)

In [6]:
tokenizer = tk.MorphologicalTokenizer(vocab_size=2000000)

tokenizer.train()

Training MorphologicalTokenizer ...


In [7]:
tokenizer.tokenize('بان الخليط وحلت الأسقام، وتثاقلت في سيرها الأيام')

['بان',
 'ال',
 '##خليط',
 'وح',
 '##لت',
 '<UNK>',
 '<UNK>',
 'في',
 'سير',
 '##ها',
 'ال',
 '##أيام']

In [8]:
tokenizer.tokenize('بان الخليط و حلت الأسقام و تثاقلت في سيرها الأيام')

['بان',
 'ال',
 '##خليط',
 'و',
 'حلت',
 '<UNK>',
 'و',
 'تثاقلت',
 'في',
 'سير',
 '##ها',
 'ال',
 '##أيام']

In [9]:
tokenizer.tokenize('أثني علَيّ بما علمتِ فإنني ... سمحٌ مخالطتي إذا لم أُظلَم')

['أثني',
 '<UNK>',
 'ب',
 '##ما',
 '<UNK>',
 'فإنن',
 '##ي',
 '<UNK>',
 '<UNK>',
 'مخالطتي',
 'إذا',
 'لم',
 '<UNK>']

In [10]:
tokenizer.tokenize('أثني علي بما علمت فإنني سمح مخالطتي إذا لم أظلم')

['أثني',
 'علي',
 'ب',
 '##ما',
 'علمت',
 'فإنن',
 '##ي',
 'سمح',
 'مخالطتي',
 'إذا',
 'لم',
 'أظلم']

## MorphologicalTokenizer with Sample

In [11]:
def window(words_seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(words_seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

def document_features(document, corpus_features):
    document_words = set(document)
    features = {}
    for word in corpus_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [12]:
import re

arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', str(text))
    return text



def remove_repeating_char(text):
    # return re.sub(r'(.)\1+', r'\1', text)     # keep only 1 repeat
    return re.sub(r'(.)\1+', r'\1\1', text)  # keep 2 repeat

def process_text(text):
    clean_text = remove_diacritics(text)
    clean_text = remove_repeating_char(clean_text)
    return tokenizer.tokenize(clean_text)

In [13]:
df_poems_sample['tokenized_poems'] = df_poems_sample.poem_text.apply(process_text)
df_poems_sample.head()

,poem_text,tokenized_poems
19455,اما ابن يعقوب فاندي الوري واعلم القوم ولا امت...,"[ا, ##ما, ابن, يعقوب, <UNK>, الور, ##ي, واع, #..."
10854,مالكثيب الحمي الي عقده مابال جرعاءه الي جرده ...,"[مالك, ##ثيب, ال, ##حمي, ال, ##ي, عقد, ##ه, ما..."
5770,اصاح اذا ما اتاك القضاء لم يقك الدرع والجوشن ...,"[<UNK>, ا, ##ذا, ما, اتا, ##ك, <UNK>, لم, يق, ..."
31463,عبث النسيم بقده فتاودا وسري الحياء بخده فتورد...,"[عبث, النس, ##ي, ##م, <UNK>, فتاو, ##دا, وس, #..."
51069,رمضان يا شهر الضياء الحر من اسر الظلام اطلق ب...,"[رمضان, يا, شهر, <UNK>, ال, ##حر, من, ا, ##سر,..."


In [14]:
df_poems_sample.head()

,poem_text,tokenized_poems
19455,اما ابن يعقوب فاندي الوري واعلم القوم ولا امت...,"[ا, ##ما, ابن, يعقوب, <UNK>, الور, ##ي, واع, #..."
10854,مالكثيب الحمي الي عقده مابال جرعاءه الي جرده ...,"[مالك, ##ثيب, ال, ##حمي, ال, ##ي, عقد, ##ه, ما..."
5770,اصاح اذا ما اتاك القضاء لم يقك الدرع والجوشن ...,"[<UNK>, ا, ##ذا, ما, اتا, ##ك, <UNK>, لم, يق, ..."
31463,عبث النسيم بقده فتاودا وسري الحياء بخده فتورد...,"[عبث, النس, ##ي, ##م, <UNK>, فتاو, ##دا, وس, #..."
51069,رمضان يا شهر الضياء الحر من اسر الظلام اطلق ب...,"[رمضان, يا, شهر, <UNK>, ال, ##حر, من, ا, ##سر,..."


In [15]:
# build vocabulary and train model
model = gensim.models.Word2Vec(
    df_poems_sample['tokenized_poems'],
    window=5,
    min_count=4,
    workers=8)

In [16]:
model.wv.most_similar(positive='بحر')

[('ابن', 0.9997038245201111),
 ('بن', 0.999703586101532),
 ('لمن', 0.9996880888938904),
 ('حب', 0.9996851682662964),
 ('لهم', 0.9996663928031921),
 ('##وي', 0.9996470212936401),
 ('##رت', 0.9996291995048523),
 ('فقد', 0.999626100063324),
 ('نور', 0.9996203780174255),
 ('بالا', 0.9996114373207092)]

In [17]:
model.wv.most_similar(positive='جمال')

[('نور', 0.9994589686393738),
 ('حول', 0.9994376301765442),
 ('بن', 0.9994372725486755),
 ('##رت', 0.9994299411773682),
 ('فت', 0.9994264245033264),
 ('فقد', 0.9994195103645325),
 ('##دا', 0.999410092830658),
 ('بحر', 0.9994022846221924),
 ('ون', 0.9993926286697388),
 ('بنا', 0.9993909001350403)]

In [18]:
model.wv.most_similar(positive='قمر')

[('ماذا', 0.9991545081138611),
 ('##وي', 0.9991365075111389),
 ('##ير', 0.9991360902786255),
 ('##مد', 0.9991323947906494),
 ('##لك', 0.9991308450698853),
 ('ماء', 0.9991303086280823),
 ('##صبح', 0.9991243481636047),
 ('روح', 0.9991234540939331),
 ('حول', 0.9991183876991272),
 ('كال', 0.9991066455841064)]

In [19]:
model.wv.most_similar(positive='سيف')

[('اك', 0.9995864629745483),
 ('كم', 0.9995831847190857),
 ('بال', 0.9995620846748352),
 ('وت', 0.9995415806770325),
 ('تحت', 0.9995320439338684),
 ('ظل', 0.9995318055152893),
 ('ليل', 0.9995264410972595),
 ('مع', 0.999495267868042),
 ('كما', 0.9994727969169617),
 ('كانت', 0.9994633197784424)]

## Word Tokenizer with Sample

In [20]:
tokenizer = tk.WordTokenizer()

tokenizer.train('data.txt')

def process_text(text):
    clean_text = remove_diacritics(text)
    clean_text = remove_repeating_char(clean_text)
    return tokenizer.tokenize(clean_text)

df_poems_sample['tokenized_poems'] = df_poems_sample.poem_text.apply(process_text)

model = gensim.models.Word2Vec(
    df_poems_sample['tokenized_poems'],
    window=5,
    min_count=4,
    workers=8)

Training WordTokenizer ...


In [21]:
model.wv.most_similar(positive='بحر')

[('بلا', 0.9997566342353821),
 ('ليل', 0.999748706817627),
 ('بني', 0.9997189044952393),
 ('فلم', 0.9996925592422485),
 ('جاء', 0.9996858835220337),
 ('العمر', 0.9996828436851501),
 ('تري', 0.9996799230575562),
 ('النور', 0.9996674656867981),
 ('القوم', 0.9996668100357056),
 ('وهل', 0.9996628761291504)]

In [22]:
model.wv.most_similar(positive='جمال')

[('الظلام', 0.9989485740661621),
 ('الطير', 0.9989356398582458),
 ('مضي', 0.9989171028137207),
 ('الضحي', 0.9988778233528137),
 ('ولما', 0.9988222122192383),
 ('شمس', 0.9988126158714294),
 ('كلها', 0.998772919178009),
 ('وبين', 0.9987707734107971),
 ('الاوطان', 0.9987590312957764),
 ('النيل', 0.9987236261367798)]

In [23]:
model.wv.most_similar(positive='قمر')

[('عليك', 0.999575138092041),
 ('متي', 0.9995589256286621),
 ('وعلي', 0.9995522499084473),
 ('وكان', 0.999547004699707),
 ('نحو', 0.9995298385620117),
 ('فمن', 0.999528706073761),
 ('فيك', 0.999521791934967),
 ('المجد', 0.9995166659355164),
 ('ايها', 0.9995105862617493),
 ('–', 0.9995008111000061)]

In [24]:
model.wv.most_similar(positive='سيف')

[('الردي', 0.9993366599082947),
 ('اصبحت', 0.999299168586731),
 ('الفتي', 0.9992502927780151),
 ('محمد', 0.9992326498031616),
 ('ويا', 0.999214768409729),
 ('صار', 0.9991844296455383),
 ('النبي', 0.9991508722305298),
 ('ليلا', 0.9991368055343628),
 ('بكل', 0.9991255402565002),
 ('وعن', 0.9991235136985779)]

## Full MorphologicalTokenizer

In [25]:
tokenizer = tk.MorphologicalTokenizer(vocab_size=500000)

tokenizer.train()

def process_text(text):
    clean_text = remove_diacritics(text)
    clean_text = remove_repeating_char(clean_text)
    return tokenizer.tokenize(clean_text)

df_poems['tokenized_poems'] = df_poems.poem_text.apply(process_text)

model = gensim.models.Word2Vec(
    df_poems['tokenized_poems'],
    window=5,
    min_count=4,
    workers=8)

Training MorphologicalTokenizer ...


In [26]:
model.wv.most_similar(positive='بحر')

[('زاخر', 0.7022616863250732),
 ('بحرا', 0.6702044606208801),
 ('خضم', 0.6064570546150208),
 ('موجه', 0.5909208655357361),
 ('بحار', 0.58272385597229),
 ('عباب', 0.5717765688896179),
 ('نهر', 0.5558062791824341),
 ('موج', 0.551268994808197),
 ('مزبد', 0.5420432090759277),
 ('نبع', 0.5362871885299683)]

In [27]:
model.wv.most_similar(positive='سيف')

[('حسام', 0.8357911705970764),
 ('صارم', 0.7844464778900146),
 ('سيفا', 0.7601202130317688),
 ('غمد', 0.7288745045661926),
 ('نصل', 0.7186676859855652),
 ('##سيف', 0.7126127481460571),
 ('رمح', 0.6803518533706665),
 ('مضاء', 0.6684632897377014),
 ('حد', 0.6606724858283997),
 ('مرهف', 0.6414411664009094)]

In [28]:
model.wv.most_similar(positive='قمر')

[('قمرا', 0.8321205377578735),
 ('كوكب', 0.8193356394767761),
 ('ضياء', 0.8184514045715332),
 ('يضيء', 0.81216961145401),
 ('مصباح', 0.7998147010803223),
 ('سراج', 0.7965234518051147),
 ('ضوء', 0.7958341836929321),
 ('نجم', 0.7855615019798279),
 ('منير', 0.7762529253959656),
 ('هلال', 0.7748417854309082)]

In [29]:
model.wv.most_similar(positive='جمال')

[('محيا', 0.758857786655426),
 ('حسن', 0.7402985692024231),
 ('منظر', 0.7312740683555603),
 ('محاسن', 0.7285208702087402),
 ('بهجة', 0.7271338105201721),
 ('حسنا', 0.7206720113754272),
 ('بديع', 0.7179510593414307),
 ('سحر', 0.7161159515380859),
 ('بهاء', 0.6947004795074463),
 ('معاني', 0.6945234537124634)]

In [30]:
model.wv.most_similar(positive='قلوب')

[('صدور', 0.6887970566749573),
 ('نفوس', 0.6794796586036682),
 ('عقول', 0.6202598810195923),
 ('عيون', 0.6193496584892273),
 ('قبور', 0.5996277928352356),
 ('جسوم', 0.5928557515144348),
 ('نحور', 0.584489643573761),
 ('نيران', 0.5829363465309143),
 ('سيوف', 0.5720204710960388),
 ('جلود', 0.5718913078308105)]

In [31]:
model.wv.most_similar(positive='لوعة')

[('جوي', 0.8924345374107361),
 ('صبابة', 0.8458849191665649),
 ('تباريح', 0.8353836536407471),
 ('غرام', 0.8140920400619507),
 ('زفرة', 0.8027799129486084),
 ('الوجد', 0.8018087148666382),
 ('حرق', 0.7945452332496643),
 ('شجو', 0.7773582935333252),
 ('حشاشة', 0.7738944888114929),
 ('لواعج', 0.7705483436584473)]

In [32]:
model.wv.most_similar(negative='سيف')

[('حياتي', 0.392706036567688),
 ('ذكري', 0.3365645110607147),
 ('##ذات', 0.32563862204551697),
 ('لذة', 0.3195958733558655),
 ('شبابي', 0.3143406808376312),
 ('ليالي', 0.3047560453414917),
 ('تذكرت', 0.2986513674259186),
 ('ليال', 0.29781585931777954),
 ('صبا', 0.2971852123737335),
 ('تلك', 0.2946537733078003)]

In [33]:
model.wv.__getitem__('سيف')

array([-0.7767816 , -0.16945663,  0.2165929 ,  0.6141618 ,  1.0802549 ,
        0.53660417, -0.39346215,  1.4651189 , -1.0488359 ,  0.55736053,
       -1.7463926 , -1.1607833 ,  1.8985958 , -0.68191653,  0.58346945,
       -0.42862454,  1.1656873 ,  0.9263713 , -0.6268776 ,  0.2798496 ,
       -1.8979998 ,  0.8955211 , -1.6486388 , -1.4201765 ,  0.3805119 ,
       -0.15150042,  0.13304031,  0.26991847,  0.47720066,  0.45403802,
        1.023573  ,  0.78011346, -1.37628   , -0.68427426,  1.5151603 ,
        1.4106754 ,  1.258872  ,  1.7807823 ,  0.74668074,  0.6710356 ,
        1.1388934 ,  0.52056897,  0.60692537,  1.6768069 ,  2.0535004 ,
        0.24564587, -1.3437184 ,  1.9841601 , -0.9223384 ,  1.0529785 ,
        0.17671028, -0.913571  , -1.57941   , -0.19650276, -2.105694  ,
        0.17080864,  0.8662784 ,  1.0394912 ,  1.3974406 ,  0.02526223,
        1.001802  , -3.0876353 , -0.99691105,  0.7273724 , -0.2023381 ,
       -0.52602834,  0.831913  ,  0.24446866, -1.5892559 ,  0.27

In [37]:
def embedPoem(token_poem):
    clear_poem = [token for token in token_poem if token != '<UNK>']
    embedded_tokens = np.array([model.wv.__getitem__(token) for token in clear_poem])
    return np.mean(embedded_tokens, axis=0)

df_poems['embedded_poems'] = df_poems['tokenized_poems'].apply(embedPoem)
df_poems['embedded_poems']

KeyError: "Key 'شرفتان' not present"